In [49]:
from icecream import ic
from ordered_set import OrderedSet
from itertools import product
import itertools as _itertools
import stim
from idtcorev2 import jacobian_coefficient_calc
from pygsti.circuits.circuit import Circuit
from pygsti.baseobjs.label import Label
import math

# Update important parameters here!
VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV

In [50]:
num_qubits = 2
max_weight = 2
term_dict = {("H", "XX"): 0.0001}

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [51]:
HS_index_iterator = stim.PauliString.iter_all(
    num_qubits, min_weight=1, max_weight=max_weight
)

pauli_node_attributes = list([p for p in HS_index_iterator])
ca_pauli_node_attributes = list(_itertools.combinations(pauli_node_attributes, 2))

def ca_pauli_weight_filter(pauli_pair, max_weight):
    used_indices_1 = set(
        i for i, ltr in enumerate(str(pauli_pair[0])[1:]) if ltr != "_"
    )
    used_indices_2 = set(
        i for i, ltr in enumerate(str(pauli_pair[1])[1:]) if ltr != "_"
    )
    union = used_indices_1.union(used_indices_2)
    if len(union) > 0 and len(union) <= max_weight:
        return True

ca_pauli_node_attributes = [
    ppair
    for ppair in ca_pauli_node_attributes
    if ca_pauli_weight_filter(ppair, max_weight)
]

measure_string_iterator = stim.PauliString.iter_all(
    num_qubits, min_weight=num_qubits
)
sign_iterator = list(product([1,-1], repeat=num_qubits))
awk_iterator = list(i[0] for i in product(sign_iterator, [p for p in measure_string_iterator]))
prep_string_iterator = product([math.prod([item for item in sign_tuple]) for sign_tuple in sign_iterator],[p for p in measure_string_iterator])
measure_string_attributes = list([p for p in measure_string_iterator])
prep_string_attributes = list(a*b for a,b in prep_string_iterator)
prep_meas_pair = list(product(prep_string_attributes, measure_string_attributes))

In [52]:
hs_error_gen_classes = "hs"
ca_error_gen_classes = "ca"


hs_experiment = list(
    product(
        hs_error_gen_classes,
        pauli_node_attributes,
        zip(awk_iterator, prep_string_attributes),
        measure_string_attributes,
    )
)
ca_experiment = list(
    product(
        ca_error_gen_classes,
        ca_pauli_node_attributes,
        zip(awk_iterator, prep_string_attributes),
        measure_string_attributes,
    )
)

In [53]:
hs_experiment

[('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+XX")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+XY")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+XZ")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+YX")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+YY")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+YZ")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+ZX")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+ZY")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XX")),
  stim.PauliString("+ZZ")),
 ('h',
  stim.PauliString("+X_"),
  ((1, 1), stim.PauliString("+XY")),
  stim.PauliString("+XX")),
 ('h',
  s

In [54]:
import pandas as pd

# df = pd.DataFrame()
jacobian_coef_dict = {"index": OrderedSet(), "columns": OrderedSet()}
data = {}

# These come back as class, index, prep_str, meas_str, observ_str: coef
# I THINK this is right, should double check and write unit tests
for key in hs_experiment + ca_experiment:
    elt = jacobian_coefficient_calc(*key)
    for el in elt:
        if el:
            observable = ",".join(str(s)[:] for s in el[-5:-1])
            sign_string = el[-4]
            egen = ",".join(str(s) for s in el[:-5])
            coef = int(el[-1])
            jacobian_coef_dict["index"].add(observable)
            jacobian_coef_dict["columns"].add(egen)
            if data.get(egen):
                data[egen].append(coef)
            else:
                data[egen] = [coef]

df = pd.DataFrame(data, index= jacobian_coef_dict["index"], columns=jacobian_coef_dict["columns"])

In [55]:
jacobian_coef_dict["columns"]
jacobian_coef_dict["index"]

OrderedSet(['+XX,(1, 1),+XX,+X_', '+XX,(1, 1),+XX,+_X', '+XX,(1, 1),+XX,+XX', '+XX,(1, 1),+XY,+X_', '+XX,(1, 1),+XY,+_Y', '+XX,(1, 1),+XY,+XY', '+XX,(1, 1),+XZ,+X_', '+XX,(1, 1),+XZ,+_Z', '+XX,(1, 1),+XZ,+XZ', '+XX,(1, 1),+YX,+Y_', '+XX,(1, 1),+YX,+_X', '+XX,(1, 1),+YX,+YX', '+XX,(1, 1),+YY,+Y_', '+XX,(1, 1),+YY,+_Y', '+XX,(1, 1),+YY,+YY', '+XX,(1, 1),+YZ,+Y_', '+XX,(1, 1),+YZ,+_Z', '+XX,(1, 1),+YZ,+YZ', '+XX,(1, 1),+ZX,+Z_', '+XX,(1, 1),+ZX,+_X', '+XX,(1, 1),+ZX,+ZX', '+XX,(1, 1),+ZY,+Z_', '+XX,(1, 1),+ZY,+_Y', '+XX,(1, 1),+ZY,+ZY', '+XX,(1, 1),+ZZ,+Z_', '+XX,(1, 1),+ZZ,+_Z', '+XX,(1, 1),+ZZ,+ZZ', '+XY,(1, 1),+XX,+X_', '+XY,(1, 1),+XX,+_X', '+XY,(1, 1),+XX,+XX', '+XY,(1, 1),+XY,+X_', '+XY,(1, 1),+XY,+_Y', '+XY,(1, 1),+XY,+XY', '+XY,(1, 1),+XZ,+X_', '+XY,(1, 1),+XZ,+_Z', '+XY,(1, 1),+XZ,+XZ', '+XY,(1, 1),+YX,+Y_', '+XY,(1, 1),+YX,+_X', '+XY,(1, 1),+YX,+YX', '+XY,(1, 1),+YY,+Y_', '+XY,(1, 1),+YY,+_Y', '+XY,(1, 1),+YY,+YY', '+XY,(1, 1),+YZ,+Y_', '+XY,(1, 1),+YZ,+_Z', '+XY,(1, 1),+YZ,+YZ'

In [56]:
df

,"H,+X_","H,+Y_","H,+Z_","H,+_X","H,+_Y","H,+_Z","H,+XX","H,+XY","H,+XZ","H,+YX",...,"A,+YY,+YZ","A,+YY,+ZX","A,+YY,+ZY","A,+YY,+ZZ","A,+YZ,+ZX","A,+YZ,+ZY","A,+YZ,+ZZ","A,+ZX,+ZY","A,+ZX,+ZZ","A,+ZY,+ZZ"
"+XX,(1, 1),+XX,+X_",0,0,0,0,0,0,0,0,0,0,...,-4,0,-4,0,0,0,-4,0,0,-4
"+XX,(1, 1),+XX,+_X",0,0,0,0,0,0,0,0,0,0,...,-4,0,-4,0,0,0,-4,0,0,-4
"+XX,(1, 1),+XX,+XX",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"+XX,(1, 1),+XY,+X_",0,0,0,0,0,0,0,0,0,0,...,-4,0,-4,0,0,0,-4,0,0,-4
"+XX,(1, 1),+XY,+_Y",0,0,0,0,0,2,0,0,2,0,...,0,2,0,0,0,0,0,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"+ZZ,(-1, -1),+ZY,+_Y",0,0,0,-2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0
"+ZZ,(-1, -1),+ZY,+ZY",0,0,0,-2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0
"+ZZ,(-1, -1),+ZZ,+Z_",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"+ZZ,(-1, -1),+ZZ,+_Z",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-4,0,0


In [57]:
import numpy as np
np.set_printoptions(precision=1, linewidth=1000)

In [58]:
import pygsti
from pygsti.extras import idletomography as idt
from pygsti.baseobjs import Label
gates = ["Gi", "Gx", "Gy", "Gcnot"]
max_lengths = [1, 2]
pspec = pygsti.processors.QubitProcessorSpec(
        num_qubits, gates, geometry="line", nonstd_gate_unitaries={(): num_qubits, "Gi": np.eye((2**num_qubits))},
        availability={"Gi": [tuple(i for i in range(num_qubits))]},
    )
mdl_target = pygsti.models.create_crosstalk_free_model(pspec)
paulidicts = idt.determine_paulidicts(mdl_target)
global_idle_string = [Label("Gi", tuple(i for i in range(num_qubits)))]
idle_experiments = idt.make_idle_tomography_list(
        num_qubits, max_lengths, paulidicts, maxweight=1, idle_string=global_idle_string
    )
idle_experiments

[Circuit([Gy:0Gy:1]Gi:0:1[Gy:0Gy:1][Gy:0Gy:1][Gy:0Gy:1]@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1[Gy:0Gy:1][Gy:0Gy:1][Gy:0Gy:1]@(0,1)),
 Circuit([Gy:0Gy:1]Gy:1Gy:1Gi:0:1[Gy:0Gy:1][Gy:0Gy:1][Gy:0Gy:1]@(0,1)),
 Circuit([Gy:0Gy:1]Gy:1Gy:1Gi:0:1Gi:0:1[Gy:0Gy:1][Gy:0Gy:1][Gy:0Gy:1]@(0,1)),
 Circuit([Gy:0Gy:1]Gy:0Gy:0Gi:0:1[Gy:0Gy:1][Gy:0Gy:1][Gy:0Gy:1]@(0,1)),
 Circuit([Gy:0Gy:1]Gy:0Gy:0Gi:0:1Gi:0:1[Gy:0Gy:1][Gy:0Gy:1][Gy:0Gy:1]@(0,1)),
 Circuit([Gy:0Gy:1][Gy:0Gy:1][Gy:0Gy:1]Gi:0:1[Gy:0Gy:1][Gy:0Gy:1][Gy:0Gy:1]@(0,1)),
 Circuit([Gy:0Gy:1][Gy:0Gy:1][Gy:0Gy:1]Gi:0:1Gi:0:1[Gy:0Gy:1][Gy:0Gy:1][Gy:0Gy:1]@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1[Gy:0Gx:1]Gy:0Gy:0@(0,1)),
 Circuit([Gy:0Gy:1]Gi:0:1Gi:0:1[Gy:0Gx:1]Gy:0Gy:0@(0,1)),
 Circuit([Gy:0Gy:1]Gy:1Gy:1Gi:0:1[Gy:0Gx:1]Gy:0Gy:0@(0,1)),
 Circuit([Gy:0Gy:1]Gy:1Gy:1Gi:0:1Gi:0:1[Gy:0Gx:1]Gy:0Gy:0@(0,1)),
 Circuit([Gy:0Gy:1]Gy:0Gy:0Gi:0:1[Gy:0Gx:1]Gy:0Gy:0@(0,1)),
 Circuit([Gy:0Gy:1]Gy:0Gy:0Gi:0:1Gi:0:1[Gy:0Gx:1]Gy:0Gy:0@(0,1)),
 Circuit([Gy:0Gy:1][Gy:0Gy:1][

In [59]:
mdl_datagen = pygsti.models.create_crosstalk_free_model(
pspec, lindblad_error_coeffs={"Gi": term_dict},lindblad_parameterization="GLND")

# Error models! Random with right CP constraints from Taxonomy paper
ds = pygsti.data.simulate_data(
    mdl_datagen, idle_experiments, 10000000, seed=8675309, sample_error="none"
)

In [60]:
from idttools import allerrors, all_full_length_observables, alloutcomes
import collections as _collections
from pygsti.circuits.circuit import Circuit as _Circuit
def report_observed_rates(nqubits,
    dataset,
    max_lengths,
    pauli_basis_dicts,
    maxweight=2,
    idle_string=global_idle_string):
    
    all_fidpairs = dict(enumerate(idt.idle_tomography_fidpairs(nqubits)))
    ic(all_fidpairs)
    if nqubits == 1:  # special case where line-labels may be ('*',)
        if len(dataset) > 0:
            first_circuit = list(dataset.keys())[0]
            line_labels = first_circuit.line_labels
        else:
            line_labels = (0,)
        GiStr = _Circuit(idle_string, line_labels=line_labels)
    else:
        GiStr = _Circuit(idle_string, num_lines=nqubits)
    obs_infos = dict()
    errors = allerrors(nqubits, maxweight)
    fit_order = 1
    observed_error_rates = {}
    obs_error_rates_by_exp = {}
    whatever = {}
    for ifp, pauli_fidpair in all_fidpairs.items():
        all_outcomes = idt.idttools.allobservables(pauli_fidpair[1], maxweight)
        infos_for_this_fidpair = _collections.OrderedDict()
        ic(pauli_fidpair)
        for j, out in enumerate(all_outcomes):
            info = idt.compute_observed_err_rate(
                dataset,
                pauli_fidpair,
                pauli_basis_dicts,
                GiStr,
                out,
                max_lengths,
                fit_order,
            )

            infos_for_this_fidpair[out] = info
            
            obs_infos[ifp] = infos_for_this_fidpair
            observed_error_rates[ifp] = [
                info["rate"] for info in infos_for_this_fidpair.values()
            ]
            obs_error_rates_by_exp[str(pauli_fidpair[0]).replace("+",""), str(pauli_fidpair[1]).replace("+",""), str(out).replace("I","_")] = [
                info["rate"] for info in infos_for_this_fidpair.values()
            ][-1]
        whatever[pauli_fidpair] = 1
    return observed_error_rates, obs_error_rates_by_exp

In [61]:
observed_error_rates, obs_error_rates_by_exp = report_observed_rates(num_qubits, ds, max_lengths, paulidicts)

ic| all_fidpairs: {0: (State[+X+X], State[+X+X]),
                   1: (State[+X-X], State[+X+X]),
                   2: (State[-X+X], State[+X+X]),
                   3: (State[-X-X], State[+X+X]),
                   4: (State[+X+X], State[+X+Y]),
                   5: (State[+X-X], State[+X+Y]),
                   6: (State[-X+X], State[+X+Y]),
                   7: (State[-X-X], State[+X+Y]),
                   8: (State[+X+X], State[+X+Z]),
                   9: (State[+X-X], State[+X+Z]),
                   10: (State[-X+X], State[+X+Z]),
                   11: (State[-X-X], State[+X+Z]),
                   12: (State[+X+X], State[+Y+X]),
                   13: (State[+X-X], State[+Y+X]),
                   14: (State[-X+X], State[+Y+X]),
                   15: (State[-X-X], State[+Y+X]),
                   16: (State[+X+X], State[+Y+Y]),
                   17: (State[+X-X], State[+Y+Y]),
                   18: (State[-X+X], State[+Y+Y]),
                   19: (State[-X-X], Stat

In [62]:
obs_rats = [v for v in obs_error_rates_by_exp.values()]

In [63]:
df.index

Index(['+XX,(1, 1),+XX,+X_', '+XX,(1, 1),+XX,+_X', '+XX,(1, 1),+XX,+XX',
       '+XX,(1, 1),+XY,+X_', '+XX,(1, 1),+XY,+_Y', '+XX,(1, 1),+XY,+XY',
       '+XX,(1, 1),+XZ,+X_', '+XX,(1, 1),+XZ,+_Z', '+XX,(1, 1),+XZ,+XZ',
       '+XX,(1, 1),+YX,+Y_',
       ...
       '+ZZ,(-1, -1),+YZ,+YZ', '+ZZ,(-1, -1),+ZX,+Z_', '+ZZ,(-1, -1),+ZX,+_X',
       '+ZZ,(-1, -1),+ZX,+ZX', '+ZZ,(-1, -1),+ZY,+Z_', '+ZZ,(-1, -1),+ZY,+_Y',
       '+ZZ,(-1, -1),+ZY,+ZY', '+ZZ,(-1, -1),+ZZ,+Z_', '+ZZ,(-1, -1),+ZZ,+_Z',
       '+ZZ,(-1, -1),+ZZ,+ZZ'],
      dtype='object', length=972)

In [64]:
obs_error_rates_by_exp

{('XX', 'XX', ' X_'): -1.6953455232808294e-16,
 ('XX', 'XX', ' _X'): -1.6953455232808294e-16,
 ('XX', 'XX', ' XX'): -1.6953455232808294e-16,
 ('X-X', 'XX', ' X_'): -1.6953455232808294e-16,
 ('X-X', 'XX', ' _X'): -1.4372227154337353e-16,
 ('X-X', 'XX', ' XX'): -1.4372227154337353e-16,
 ('-XX', 'XX', ' X_'): -1.4372227154337353e-16,
 ('-XX', 'XX', ' _X'): -1.6953455232808294e-16,
 ('-XX', 'XX', ' XX'): -1.4372227154337353e-16,
 ('-X-X', 'XX', ' X_'): -1.4372227154337353e-16,
 ('-X-X', 'XX', ' _X'): -1.4372227154337353e-16,
 ('-X-X', 'XX', ' XX'): -1.6953455232808294e-16,
 ('XX', 'XY', ' X_'): -1.6953455232808294e-16,
 ('XX', 'XY', ' _Y'): 0.0,
 ('XX', 'XY', ' XY'): 0.0,
 ('X-X', 'XY', ' X_'): -1.6953455232808294e-16,
 ('X-X', 'XY', ' _Y'): 0.0,
 ('X-X', 'XY', ' XY'): 0.0,
 ('-XX', 'XY', ' X_'): -1.4372227154337353e-16,
 ('-XX', 'XY', ' _Y'): 0.0,
 ('-XX', 'XY', ' XY'): 0.0,
 ('-X-X', 'XY', ' X_'): -1.4372227154337353e-16,
 ('-X-X', 'XY', ' _Y'): 0.0,
 ('-X-X', 'XY', ' XY'): 0.0,
 ('XX', 

In [65]:
j = df.to_numpy()

In [66]:
np.linalg.matrix_rank(j)

150

In [67]:
jinv = np.linalg.pinv(j)

In [68]:
intrins_errs = jinv @ obs_rats

# Supposed intrinsic error rates (wrong)

In [69]:
dict(zip(df.columns,intrins_errs))

{'H,+X_': -6.055321365233851e-08,
 'H,+Y_': 1.0468954210058765e-05,
 'H,+Z_': 7.884219203000718e-07,
 'H,+_X': -3.5314372433521682e-06,
 'H,+_Y': -5.41210753303847e-06,
 'H,+_Z': 6.105239430446388e-07,
 'H,+XX': -2.7620205840024523e-06,
 'H,+XY': -9.698977965691363e-07,
 'H,+XZ': 5.674554111173429e-07,
 'H,+YX': 1.3724302701730491e-06,
 'H,+YY': -1.3837349858570815e-06,
 'H,+YZ': -3.384101735140816e-08,
 'H,+ZX': 7.43790897231185e-07,
 'H,+ZY': -3.77854431268356e-07,
 'H,+ZZ': 1.0209118198544274e-06,
 'S,+X_': -5.602974311630189e-06,
 'S,+Y_': -5.918742391287906e-07,
 'S,+Z_': 3.4802822856014463e-06,
 'S,+_X': 3.4235605688335455e-06,
 'S,+_Y': -3.514642789019975e-06,
 'S,+_Z': -4.168173783088509e-07,
 'S,+XX': -9.120071479541471e-07,
 'S,+XY': 5.061117337792347e-06,
 'S,+XZ': -1.6925448599907872e-07,
 'S,+YX': -4.154081761726234e-06,
 'S,+YY': -5.8916212963050545e-08,
 'S,+YZ': 2.7763279051253498e-06,
 'S,+ZX': 5.86262370032128e-07,
 'S,+ZY': -3.286029810803337e-06,
 'S,+ZZ': -2.118273